In [1]:
import torch, sys
sys.path.append('/home/dgotzens/scripts')
from timedataparser import load_all
import pdfdefaults as pdf
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter
from matplotlib.dates import DateFormatter
import tools

date='23-10-09'
# date = '24-02-16'
header,time,data,temperature,reboots = load_all(f'/home/dgotzens/recording/{date}/', True)
M,K,L = data.shape
# maxdist = 50
maxdist = 25
torch.Tensor.dB = lambda self: 20*(self).log10()
torch.Tensor.dBr = lambda self: (self/self.max()).dB()


loaded data with shape torch.Size([1022, 192, 983])


In [2]:
N=2**14
drift = torch.empty((K,L),dtype=torch.cfloat)
nmin, nmax = int(1.4*N/maxdist), int(2*N/maxdist)
window = torch.hann_window(M)
fft = torch.fft.fft(window[:,None,None]*data, n=N, dim=0)
n = fft.mean(1).abs()[nmin:nmax,:].argmax(0) + nmin
for l in range(L):
    drift[:,l] = fft[n[l],:,l] 
drift /= fft[n[0],:,0,None]

In [3]:
Ntx,Nrx = 12,16
driftmatrix = drift.unflatten(0, (Ntx,Nrx))**2
txdrift = driftmatrix[:,0,:]
rxdrift = 10**(driftmatrix/txdrift[:,None,:]).log10().mean(0)

pdf.setup()
fig, ax = plt.subplots(2,2, sharex=True)
txstyles = [f'{col}{linestyle}' for col in 'bgrk' for linestyle in ('-','-.','--')]
for id, style in enumerate(txstyles):
    ax[0,0].plot(time[::10], txdrift[id,::10].abs(), style)
    ax[1,0].plot(time[::10], tools.unwrap(txdrift[id,::10].angle())*180/torch.pi, style)

ax[0,0].set_title('Tx-Array\namplitude')
ax[1,0].set_title('phase')

ax[1,0].yaxis.set_major_formatter(EngFormatter('°'))
ax[1,0].xaxis.set_major_formatter(DateFormatter('%H:%M'))
plt.setp(ax[1,0].xaxis.get_majorticklabels(), rotation=90)

rxstyles = [f'{col}{linestyle}' for col in 'bgrk' for linestyle in ('-','-.','--',':')]
for id, style in enumerate(rxstyles):
    ax[0,1].plot(time[::10], rxdrift[id,::10].abs(), style)
    phase = tools.unwrap(rxdrift[id,::10].angle())*180/torch.pi
    ax[1,1].plot(time[::10], phase, style, label=f'chip{id//4}, ant{id%4}')
ax[0,1].set_title('Rx-Array\namplitude')
ax[1,1].set_title('phase')
ax[1,1].xaxis.set_major_formatter(DateFormatter('%H:%M'))
ax[1,1].yaxis.set_major_formatter(EngFormatter('°'))
plt.setp(ax[1,1].xaxis.get_majorticklabels(), rotation=90)
for a in ax.flat:
    a.grid()
fig.legend(*ax[1,1].get_legend_handles_labels(), bbox_to_anchor=(0.5,0), loc='upper center', ncol=4)
fig.set_size_inches(1*pdf.a4_textwidth,0.8*pdf.a4_textwidth)
fig.savefig('/home/dgotzens/thesis/figures/antenna_drift.pdf', bbox_inches='tight')